## Solveur SAT

Télécharger glophersat.zip sur Edunao et le déziper.

In [80]:
#Set 
N = 3 #nombre de matière
s = {x for x in range(N)}

In [81]:
#Variable aplha
alpha = []
for i in range(0,N):
    for k in range(0,21):
        alpha.append((i,k)) # ==> donne tous les alpha i k 

#Dictionnaire alpha
v2i_alpha = {v : i+1 for i,v in enumerate(alpha)} # à chaque variable associe un nombre
A = len(v2i_alpha)


In [82]:
from itertools import chain
from itertools import combinations

#Créer l'ensemble des subset 
def powerset(iterable): 
    s = list(iterable)
    return( chain.from_iterable(combinations(s, r) for r in range(len(s)+1)) )

#L'ensemble des subset 
subsets = list(powerset(s)) 
print('subsets')
print(subsets)

#Dictionnaire beta
v2i_beta = {v : A+i+1 for i,v in enumerate(subsets)}
print('v2i_beta')
print(v2i_beta)

subsets
[(), (0,), (1,), (2,), (0, 1), (0, 2), (1, 2), (0, 1, 2)]
v2i_beta
{(): 64, (0,): 65, (1,): 66, (2,): 67, (0, 1): 68, (0, 2): 69, (1, 2): 70, (0, 1, 2): 71}


In [83]:
#Clause 1
clause_1 = []

for i in range(0,N):
    for k in range(0,21):
        for j in range(k+1,21):
            clause_1.append([-v2i_alpha[(i,k)], v2i_alpha[(i,j)]])

#print(clause_1)
         

In [84]:
#Clause 2
clause_2 = []

for i in subsets:
    C_prime = set(i)
    for j in subsets:
        C = set(j)
        if C.issubset(C_prime):
            clause_2.append([-v2i_beta[j], v2i_beta[i]])

#print(clause_2)

In [85]:
#Clause 3

clause_3 = []

for c in subsets:
    C = set(c)
    D = s.difference(C)
    j = tuple(D)
    alpha = []
    for k in range(0,21):
        for i in range(0,N):
            if i in C:
                alpha.append(v2i_alpha[(i,k)])
    
    
    clause_3.append(alpha+[v2i_beta[j]])
            
print(clause_3)

[[71], [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 70], [22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 69], [43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 68], [1, 22, 2, 23, 3, 24, 4, 25, 5, 26, 6, 27, 7, 28, 8, 29, 9, 30, 10, 31, 11, 32, 12, 33, 13, 34, 14, 35, 15, 36, 16, 37, 17, 38, 18, 39, 19, 40, 20, 41, 21, 42, 67], [1, 43, 2, 44, 3, 45, 4, 46, 5, 47, 6, 48, 7, 49, 8, 50, 9, 51, 10, 52, 11, 53, 12, 54, 13, 55, 14, 56, 15, 57, 16, 58, 17, 59, 18, 60, 19, 61, 20, 62, 21, 63, 66], [22, 43, 23, 44, 24, 45, 25, 46, 26, 47, 27, 48, 28, 49, 29, 50, 30, 51, 31, 52, 32, 53, 33, 54, 34, 55, 35, 56, 36, 57, 37, 58, 38, 59, 39, 60, 40, 61, 41, 62, 42, 63, 65], [1, 22, 43, 2, 23, 44, 3, 24, 45, 4, 25, 46, 5, 26, 47, 6, 27, 48, 7, 28, 49, 8, 29, 50, 9, 30, 51, 10, 31, 52, 11, 32, 53, 12, 33, 54, 13, 34, 55, 14, 35, 56, 15, 36, 57, 16, 37, 58, 17, 38, 59, 18, 39, 60, 19, 40, 61, 20, 41, 

In [86]:
clause_4 = []

for c in subsets:
    C = set(c)
    j = tuple(C)
    alpha = []
    for k in range(0,21):
        for i in range(0,N):
            if i in C:
                alpha.append(-v2i_alpha[(i,k)])
    
    clause_4.append(alpha+[-v2i_beta[j]])

print(clause_4)

[[-64], [-1, -2, -3, -4, -5, -6, -7, -8, -9, -10, -11, -12, -13, -14, -15, -16, -17, -18, -19, -20, -21, -65], [-22, -23, -24, -25, -26, -27, -28, -29, -30, -31, -32, -33, -34, -35, -36, -37, -38, -39, -40, -41, -42, -66], [-43, -44, -45, -46, -47, -48, -49, -50, -51, -52, -53, -54, -55, -56, -57, -58, -59, -60, -61, -62, -63, -67], [-1, -22, -2, -23, -3, -24, -4, -25, -5, -26, -6, -27, -7, -28, -8, -29, -9, -30, -10, -31, -11, -32, -12, -33, -13, -34, -14, -35, -15, -36, -16, -37, -17, -38, -18, -39, -19, -40, -20, -41, -21, -42, -68], [-1, -43, -2, -44, -3, -45, -4, -46, -5, -47, -6, -48, -7, -49, -8, -50, -9, -51, -10, -52, -11, -53, -12, -54, -13, -55, -14, -56, -15, -57, -16, -58, -17, -59, -18, -60, -19, -61, -20, -62, -21, -63, -69], [-22, -43, -23, -44, -24, -45, -25, -46, -26, -47, -27, -48, -28, -49, -29, -50, -30, -51, -31, -52, -32, -53, -33, -54, -34, -55, -35, -56, -36, -57, -37, -58, -38, -59, -39, -60, -40, -61, -41, -62, -42, -63, -70], [-1, -22, -43, -2, -23, -44, -3,

In [87]:
#Construction du DIMCS et Résolution
i2v = {}

for i in range(len(v2i_alpha)):
    i2v[i+1] = list(v2i_alpha.keys())[list(v2i_alpha.values()).index(i+1)]

for i in range(len(v2i_beta)):
    i2v[i+1+A] = list(v2i_beta.keys())[list(v2i_beta.values()).index(i+1+A)]

import subprocess

def clauses_to_dimacs(clauses,numvar) :
    dimacs = 'c This is it\np cnf '+str(numvar)+' '+str(len(clauses))+'\n'
    for clause in clauses :
        for atom in clause :
            dimacs += str(atom) +' '
        dimacs += '0\n'
    return dimacs

def write_dimacs_file(dimacs, filename):
    with open(filename, "w", newline="") as cnf:
        cnf.write(dimacs)

#Attention à utiliser la vesion du solveur compatible avec votre système d'exploitation, mettre le solveur dans le même dossier que ce notebook        

def exec_gophersat(filename, cmd = "./gophersat.exe", encoding = "utf8") :
    result = subprocess.run([cmd, filename], stdout=subprocess.PIPE, check=True, encoding=encoding)
    string = str(result.stdout)
    lines = string.splitlines()

    if lines[1] != "s SATISFIABLE":
        return False, [], {}

    model = lines[2][2:].split(" ")

    return True, [int(x) for x in model if int(x) != 0], { i2v[abs(int(v))] : int(v) > 0 for v in model if int(v)!=0} 


In [88]:
#Lancer la résolution

myClauses= clause_1 + clause_2 + clause_3 + clause_4
myDimacs = clauses_to_dimacs(myClauses,len(v2i_alpha)+len(v2i_beta))

write_dimacs_file(myDimacs,"./workingfile.cnf")
res = exec_gophersat("./workingfile.cnf")

#Résultat
print(res)

(True, [-1, -2, -3, -4, -5, -6, -7, -8, -9, -10, -11, -12, -13, -14, -15, -16, -17, -18, -19, -20, 21, -22, -23, -24, -25, -26, -27, -28, -29, -30, -31, -32, -33, -34, -35, -36, -37, -38, -39, -40, -41, 42, -43, -44, -45, -46, -47, -48, -49, -50, -51, -52, -53, -54, -55, -56, -57, -58, -59, -60, -61, -62, 63, -64, -65, -66, -67, -68, -69, -70, 71], {(0, 0): False, (0, 1): False, (0, 2): False, (0, 3): False, (0, 4): False, (0, 5): False, (0, 6): False, (0, 7): False, (0, 8): False, (0, 9): False, (0, 10): False, (0, 11): False, (0, 12): False, (0, 13): False, (0, 14): False, (0, 15): False, (0, 16): False, (0, 17): False, (0, 18): False, (0, 19): False, (0, 20): True, (1, 0): False, (1, 1): False, (1, 2): False, (1, 3): False, (1, 4): False, (1, 5): False, (1, 6): False, (1, 7): False, (1, 8): False, (1, 9): False, (1, 10): False, (1, 11): False, (1, 12): False, (1, 13): False, (1, 14): False, (1, 15): False, (1, 16): False, (1, 17): False, (1, 18): False, (1, 19): False, (1, 20): True

SAT Solver Multi classes

In [89]:
import sys
sys.path.append('../')

from generator import GradesGenerator

In [90]:
size = 100
nb_grades = 5
noise = 0
nb_class = 5

s = {i for i in range(nb_grades)}

gen = GradesGenerator(size=size, nb_grades=nb_grades,noise=noise, nb_class=nb_class)

grades, admissions = gen.generate_grades()

In [104]:
#Variable aplha
alpha = []
for i in range(nb_grades):
    for k in range(21):
        for h in range(nb_class):
            alpha.append((i,k,h)) # ==> donne tous les alpha i k h

#Dictionnaire alpha
v2i_alpha = {v : i+1 for i,v in enumerate(alpha)} # à chaque variable associe un nombre
A = len(v2i_alpha)


In [105]:
from itertools import chain
from itertools import combinations

#Créer l'ensemble des subset 
def powerset(iterable): 
    s = list(iterable)
    return( chain.from_iterable(combinations(s, r) for r in range(len(s)+1)) )

#L'ensemble des subset 
subsets = list(powerset(s)) 


#Dictionnaire beta
v2i_beta = {v : A+i+1 for i,v in enumerate(subsets)}


In [106]:
clause_1 = []

for i in range(nb_grades):
    for k in range(21):
        for h in range(nb_class):
            for j in range(k+1,21):
                clause_1.append([-v2i_alpha[(i,k,h)], v2i_alpha[(i,j,h)]])

In [107]:
clause_2 = []

for i in subsets:
    C_prime = set(i)
    for j in subsets:
        C = set(j)
        if C.issubset(C_prime):
            clause_2.append([-v2i_beta[j], v2i_beta[i]])

In [110]:
clause_3 = []

for c in subsets:
    C = set(c)
    D = s.difference(C)
    j = tuple(D)
    alpha = []
    for student in grades:
        for h in range(nb_class):
            for i in range(0,nb_grades):
                if i in C:
                    alpha.append(v2i_alpha[(i,student[i],h)])

    
        clause_3.append(alpha+[v2i_beta[j]])

In [111]:
clause_4 = []

for c in subsets:
    C = set(c)
    j = tuple(C)
    alpha = []
    for student in grades:
        for h in range(nb_class):
            for i in range(0,nb_grades):
                if i in C:
                    alpha.append(-v2i_alpha[(i,student[i],h)])
    
        clause_4.append(alpha+[-v2i_beta[j]])


In [112]:
clause_5 = []

for k in range(0,21):
    for i in range(nb_grades):
        for h in range(nb_class):
            for j in range(h+1, nb_class):
                clause_5.append([v2i_alpha[(i,k,h)],-v2i_alpha[(i,k,j)]])

In [113]:
#Construction du DIMCS et Résolution
i2v = {}

for i in range(len(v2i_alpha)):
    i2v[i+1] = list(v2i_alpha.keys())[list(v2i_alpha.values()).index(i+1)]

for i in range(len(v2i_beta)):
    i2v[i+1+A] = list(v2i_beta.keys())[list(v2i_beta.values()).index(i+1+A)]

import subprocess

def clauses_to_dimacs(clauses,numvar) :
    dimacs = 'c This is it\np cnf '+str(numvar)+' '+str(len(clauses))+'\n'
    for clause in clauses :
        for atom in clause :
            dimacs += str(atom) +' '
        dimacs += '0\n'
    return dimacs

def write_dimacs_file(dimacs, filename):
    with open(filename, "w", newline="") as cnf:
        cnf.write(dimacs)

#Attention à utiliser la vesion du solveur compatible avec votre système d'exploitation, mettre le solveur dans le même dossier que ce notebook        

def exec_gophersat(filename, cmd = "./gophersat.exe", encoding = "utf8") :
    result = subprocess.run([cmd, filename], stdout=subprocess.PIPE, check=True, encoding=encoding)
    string = str(result.stdout)
    lines = string.splitlines()

    if lines[1] != "s SATISFIABLE":
        return False, [], {}

    model = lines[2][2:].split(" ")

    return True, [int(x) for x in model if int(x) != 0], { i2v[abs(int(v))] : int(v) > 0 for v in model if int(v)!=0} 


In [114]:
myClauses= clause_1 + clause_2 + clause_3 + clause_4 + clause_5
myDimacs = clauses_to_dimacs(myClauses,len(v2i_alpha)+len(v2i_beta))

write_dimacs_file(myDimacs,"./workingfile_multi.cnf")
res = exec_gophersat("./workingfile_multi.cnf")

#Résultat
print(res)

(True, [-1, -2, -3, -4, -5, -6, -7, -8, -9, -10, -11, -12, -13, -14, -15, -16, -17, -18, -19, -20, -21, -22, -23, -24, -25, -26, -27, -28, -29, -30, -31, -32, -33, -34, -35, -36, -37, -38, -39, -40, -41, -42, -43, -44, -45, -46, -47, -48, -49, -50, -51, -52, -53, -54, 55, -56, -57, 58, -59, -60, 61, -62, -63, -64, -65, -66, -67, -68, -69, -70, -71, -72, -73, -74, -75, -76, -77, -78, -79, -80, -81, -82, -83, -84, -85, -86, -87, -88, -89, -90, -91, -92, -93, -94, -95, -96, -97, -98, -99, -100, -101, -102, -103, -104, -105, -106, -107, -108, -109, -110, -111, -112, -113, -114, -115, -116, -117, 118, -119, -120, 121, -122, -123, 124, -125, -126, -127, -128, -129, -130, -131, -132, -133, -134, -135, -136, -137, -138, -139, -140, -141, -142, -143, -144, -145, -146, -147, -148, -149, -150, -151, -152, -153, -154, -155, -156, -157, -158, -159, -160, -161, -162, -163, -164, -165, -166, -167, -168, -169, -170, -171, -172, -173, -174, -175, -176, -177, 178, -179, -180, 181, -182, -183, 184, -185,